## Import architectures and necessary libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from densenet import DenseNet
from glob import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

2023-05-10 05:11:23.622922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 05:11:24.192197: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Preprocessing images for train model

In [2]:
labels = []
images = []

imagesfiles = [i for i in glob('data/*.JPEG')]
for i in imagesfiles:
    image = cv2.imread(i)/255.0
    image = cv2.resize(image, (224,224))
    images.append(image)
images = np.array(images)

with open('data/imagenet_2012_validation_synset_labels.txt') as f:
    labelstxt = f.read().split('\n')[:-1]
with open('data/labels.txt') as f:
    labs = f.read()
    
for i in labelstxt:
    bosh = labs.find(i)
    labels.append(int(labs[bosh+10:bosh+10+labs[bosh+10:].find(' ')])-1)
    
images = np.array(images)
labels = np.array(labels)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=100)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(40000, 224, 224, 3) (40000,)
(10000, 224, 224, 3) (10000,)


## Build model

In [4]:
class Model(keras.models.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.base_model = DenseNet()
    def call(self, x):
        x = self.base_model(x)
        return x

## Define loss function and optimizer

In [5]:
model = Model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2023-05-09 19:04:41.769632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 19:04:41.769864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 19:04:41.794532: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the 

## Training model

In [6]:
model.fit(x_train, y_train, batch_size=16, epochs=5, validation_split=0.2)

Epoch 1/5


2023-05-09 19:04:46.695226: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584000 exceeds 10% of free system memory.


2000/2000 [==============================] - 3576s 2s/step - loss: 6.9523 - accuracy: 5.3125e-04 - val_loss: 6.9442 - val_accuracy: 6.2500e-04
Epoch 2/5
2000/2000 [==============================] - 3532s 2s/step - loss: 6.9131 - accuracy: 0.0011 - val_loss: 6.9369 - val_accuracy: 5.0000e-04
Epoch 3/5
 582/2000 [=======>......................] - ETA: 40:02 - loss: 6.9079 - accuracy: 9.6649e-04

KeyboardInterrupt: 

## Finetune architecture

In [2]:
train = tfds.load('oxford_iiit_pet', split='train')
test = tfds.load('oxford_iiit_pet', split='test')

x_train, y_train, y_train2, x_test, y_test, y_test2 = [], [], [], [], [], []
for i in train:
    a = i['image'].numpy()
    x_train.append(cv2.resize(a, (224, 224))/255.0)
    y_train.append(i['label'].numpy())
    y_train2.append(i['species'].numpy())
    
for i in test:
    a = i['image'].numpy()
    x_test.append(cv2.resize(a, (224, 224))/255.0)
    y_test.append(i['label'].numpy())
    y_test2.append(i['species'].numpy())
    
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train2 = np.array(y_train2)
y_train2 = np.expand_dims(y_train2, axis=1)

x_test = np.array(x_test)
y_test = np.array(y_test)
y_test2 = np.array(y_test2)
y_test2 = np.expand_dims(y_test2, axis=1)

x_train.shape, y_train.shape, y_train2.shape, x_test.shape, y_test.shape, y_test2.shape

2023-05-10 05:11:26.729342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 05:11:26.729645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 05:11:26.774482: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the 

((3680, 224, 224, 3),
 (3680,),
 (3680, 1),
 (3669, 224, 224, 3),
 (3669,),
 (3669, 1))

In [3]:
# classification for two classes
class Model1(keras.models.Model):
    def __init__(self):
        super(Model1, self).__init__()
        self.base_model = DenseNet(include_top=False)
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(1000, activation='relu')
        self.fc2 = keras.layers.Dense(256, activation='relu')
        self.out = keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        return x
    
# classification for 37 classes
class Model2(keras.models.Model):
    def __init__(self):
        super(Model2, self).__init__()
        self.base_model = DenseNet(include_top=False)
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(1000, activation='relu')
        self.fc2 = keras.layers.Dense(256, activation='relu')
        self.out = keras.layers.Dense(37, activation='softmax')
        
    def call(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        return x
    
model1 = Model1()
model2 = Model2()

In [4]:
# loss and optimizer
loss_object1 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss1 = tf.keras.metrics.Mean(name='train_loss')
train_accuracy1 = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss1 = tf.keras.metrics.Mean(name='test_loss')
test_accuracy1 = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

# optimize functions
@tf.function
def train_step1(images, labels):
    with tf.GradientTape() as tape:
        predictions = model1(images, training=True)
        loss = loss_object1(labels, predictions)
    gradients = tape.gradient(loss, model1.trainable_variables)
    optimizer1.apply_gradients(zip(gradients, model1.trainable_variables))

    train_loss1(loss)
    train_accuracy1(labels, predictions)

@tf.function
def test_step1(images, labels):
    predictions = model1(images, training=False)
    t_loss = loss_object1(labels, predictions)
    
    test_loss1(t_loss)
    test_accuracy1(labels, predictions)

In [5]:
# loss and optimizer
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss2 = tf.keras.metrics.Mean(name='train_loss2')
train_accuracy2 = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy2')

test_loss2 = tf.keras.metrics.Mean(name='test_loss2')
test_accuracy2 = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy2')

# optimize functions
@tf.function
def train_step2(images, labels):
    with tf.GradientTape() as tape:
        predictions = model2(images, training=True)
        loss = loss_object2(labels, predictions)
    gradients = tape.gradient(loss, model2.trainable_variables)
    optimizer2.apply_gradients(zip(gradients, model2.trainable_variables))

    train_loss2(loss)
    train_accuracy2(labels, predictions)

@tf.function
def test_step2(images, labels):
    predictions = model2(images, training=False)
    t_loss = loss_object2(labels, predictions)
    
    test_loss2(t_loss)
    test_accuracy2(labels, predictions)

In [7]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

# for 2 classes
train_gen1 = DataGenerator(x_train, y_train2, 32)
test_gen1 = DataGenerator(x_test, y_test2, 32)

# for 37 classes
train_gen2 = DataGenerator(x_train, y_train, 32)
test_gen2 = DataGenerator(x_test, y_test, 32)

### Train 2 classes model

In [13]:
epochs = 15

for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss1.reset_states()
    train_accuracy1.reset_states()
    test_loss1.reset_states()
    test_accuracy1.reset_states()

    for train_images, train_labels in train_gen1:
        train_step1(train_images, train_labels)

    for test_images, test_labels in test_gen1:
        test_step1(test_images, test_labels)
    if epoch % 1 == 0:
        print(
          f'Epoch {epoch}, '
          f'Loss: {train_loss1.result()}, '
          f'Accuracy: {train_accuracy1.result() * 100}%, '
          f'Test Loss: {test_loss1.result()}, '
          f'Test Accuracy: {test_accuracy1.result() * 100}%'
      )

/home/airi/anaconda3/lib/python3.9/site-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0, Loss: 2.407275676727295, Accuracy: 60.380435943603516%, Test Loss: 0.6294675469398499, Test Accuracy: 67.75688171386719%
Epoch 1, Loss: 0.6554157137870789, Accuracy: 66.41304779052734%, Test Loss: 0.6284439563751221, Test Accuracy: 67.75688171386719%
Epoch 2, Loss: 0.6224454641342163, Accuracy: 67.98912811279297%, Test Loss: 0.6413689851760864, Test Accuracy: 66.36685943603516%
Epoch 3, Loss: 0.6004112958908081, Accuracy: 68.72282409667969%, Test Loss: 0.6910366415977478, Test Accuracy: 56.5549201965332%
Epoch 4, Loss: 0.5863730907440186, Accuracy: 70.10869598388672%, Test Loss: 0.576470673084259, Test Accuracy: 69.25592803955078%
Epoch 5, Loss: 0.5707728266716003, Accuracy: 71.1684799194336%, Test Loss: 0.6412765979766846, Test Accuracy: 70.89125061035156%
Epoch 6, Loss: 0.5552626848220825, Accuracy: 72.66304779052734%, Test Loss: 0.6390237808227539, Test Accuracy: 71.2728271484375%
Epoch 7, Loss: 0.5302475690841675, Accuracy: 73.8315200805664%, Test Loss: 0.5826123356819153,

### Train 37 classes model

In [8]:
epochs = 15

for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss2.reset_states()
    train_accuracy2.reset_states()
    test_loss2.reset_states()
    test_accuracy2.reset_states()

    for train_images, train_labels in train_gen2:
        train_step2(train_images, train_labels)

    for test_images, test_labels in test_gen2:
        test_step2(test_images, test_labels)
    if epoch % 1 == 0:
        print(
          f'Epoch {epoch}, '
          f'Loss: {train_loss2.result()}, '
          f'Accuracy: {train_accuracy2.result() * 100}%, '
          f'Test Loss: {test_loss2.result()}, '
          f'Test Accuracy: {test_accuracy2.result() * 100}%'
      )

/home/airi/anaconda3/lib/python3.9/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0, Loss: 6.428648471832275, Accuracy: 3.6141304969787598%, Test Loss: 3.6413559913635254, Test Accuracy: 2.752793788909912%
Epoch 1, Loss: 3.526376247406006, Accuracy: 7.418478488922119%, Test Loss: 3.7205257415771484, Test Accuracy: 3.515944242477417%
Epoch 2, Loss: 3.4020886421203613, Accuracy: 8.614130020141602%, Test Loss: 3.6805684566497803, Test Accuracy: 5.342055320739746%
Epoch 3, Loss: 3.293243408203125, Accuracy: 11.875%, Test Loss: 3.7665438652038574, Test Accuracy: 6.759335041046143%
Epoch 4, Loss: 3.1581883430480957, Accuracy: 14.565217971801758%, Test Loss: 3.523942232131958, Test Accuracy: 8.394658088684082%
Epoch 5, Loss: 2.9766478538513184, Accuracy: 18.478260040283203%, Test Loss: 3.4765403270721436, Test Accuracy: 9.484872817993164%
Epoch 6, Loss: 2.78430438041687, Accuracy: 22.6358699798584%, Test Loss: 3.487335681915283, Test Accuracy: 11.610793113708496%
Epoch 7, Loss: 2.550723075866699, Accuracy: 28.75%, Test Loss: 4.254498481750488, Test Accuracy: 10.68411